# Chapter 1
## Creating Tensors
First, we define a helper function, `describe(x)`, that will summarize various properties of a tensor
x, such as the type of the tensor, the dimensions of the tensor, and the contents of the tensor:

In [3]:
def describe(x):
    print("Type: {}".format(x.type()))
    print("Shape/size: {}".format(x.shape))
    print("Values: \n{}".format(x))

**PyTorch** allows us to create tensors in many different ways using the torch package. One way to
create a tensor is to initialize a random one by specifying its dimensions, as shown in this example

In [5]:
import torch
describe(torch.Tensor(2, 2))

Type: torch.FloatTensor
Shape/size: torch.Size([2, 2])
Values: 
tensor([[-2.4890e+24,  3.0728e-41],
        [ 1.4013e-45,  0.0000e+00]])


In [7]:
describe(torch.randn(3, 3))

Type: torch.FloatTensor
Shape/size: torch.Size([3, 3])
Values: 
tensor([[-0.9061, -0.2948,  0.4026],
        [-0.5223,  0.1452, -0.6678],
        [ 0.6967,  1.1662, -0.2058]])


We can also create tensors all filled with the same scalar. For creating a tensor of zeros or ones, we have built-in functions, and for filling it with specific values, we can use the `fill_()` method. Any **PyTorch** method with an underscore (_) refers to an in-place operation; that is, it modifies the
content in place without creating a new object, as shown in this example.

In [8]:
x = torch.zeros(2, 2)

describe(x)

x.fill_(5)

describe(x)

Type: torch.FloatTensor
Shape/size: torch.Size([2, 2])
Values: 
tensor([[0., 0.],
        [0., 0.]])
Type: torch.FloatTensor
Shape/size: torch.Size([2, 2])
Values: 
tensor([[5., 5.],
        [5., 5.]])


The values can either come from a list, as in the preceding example, or from a NumPy array. And, of
course, we can always go from a PyTorch tensor to a NumPy array, as well. Notice that the type of the
tensor is DoubleTensor instead of the default FloatTensor (see the next section). This
corresponds with the data type of the NumPy random matrix, a float64, as presented in this example:

In [11]:
x = torch.Tensor([[2, 3], [4, 5]])
describe(x)

Type: torch.FloatTensor
Shape/size: torch.Size([2, 2])
Values: 
tensor([[2., 3.],
        [4., 5.]])


In [14]:
import numpy as np

describe(torch.from_numpy(np.random.randn(3, 3)))

Type: torch.DoubleTensor
Shape/size: torch.Size([3, 3])
Values: 
tensor([[ 1.0106, -1.1592, -0.3613],
        [-1.2184,  0.3578,  0.1281],
        [ 1.9778, -2.0724, -0.3050]], dtype=torch.float64)


We can observe that there are different types of tensors: `torch.FloatTensor` and `torch.DoubleTensor`. They correspond to different representations of floating point numbers.

## Operations

In [19]:

x = torch.randn(2, 2)

x

tensor([[-1.4500, -0.4547],
        [-1.2337, -1.1671]])

Sum

In [20]:
x + x

tensor([[-2.8999, -0.9093],
        [-2.4675, -2.3343]])

Elementwise product

In [21]:
x * x

tensor([[2.1024, 0.2067],
        [1.5221, 1.3622]])

Matrix multiplication

In [22]:
x @ x

tensor([[2.6633, 1.1899],
        [3.2288, 1.9231]])

Resizing


In [24]:
x = torch.arange(10)

x.view(2, 5)

tensor([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]])

## Tensors and Computational Graphs

`PyTorch` tensor class encapsulates the data (the tensor itself) and a range of operations, such as
algebraic operations, indexing, and reshaping operations. However, as shown in the following, when
the requires_grad Boolean flag is set to True on a tensor, bookkeeping operations are enabled
that can track the gradient at the tensor as well as the gradient function, both of which are needed to
facilitate the gradient­based learning discussed in The Supervised Learning Paradigm”.

In [39]:
import torch
x = torch.ones(2, 2, requires_grad=True)
describe(x)
print(x.grad is None)

Type: torch.FloatTensor
Shape/size: torch.Size([2, 2])
Values: 
tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
True


In [40]:
y = (x + 2) * (x + 5) + 3
describe(y)
print(x.grad is None)

Type: torch.FloatTensor
Shape/size: torch.Size([2, 2])
Values: 
tensor([[21., 21.],
        [21., 21.]], grad_fn=<AddBackward0>)
True


In [41]:
z = y.mean()
describe(z)
print(z.grad is None)
z.backward()
print(x.grad is None)

Type: torch.FloatTensor
Shape/size: torch.Size([])
Values: 
21.0
True
False


When you create a tensor with requires_grad=True, you are requiring PyTorch to manage bookkeeping information that computes gradients. First, PyTorch will keep track of the values of the forward pass. Then, at the end of the computations, a single scalar is used to compute a backward pass. The backward pass is initiated by using the backward() method on a tensor resulting from the evaluation of a loss function. The backward pass computes a gradient value for a tensor object that participated in the forward pass.

In general, the gradient is a value that represents the slope of a function output with respect to the function input. In the computational graph setting, gradients exist for each parameter in the model and can be thought of as the parameter’s contribution to the error signal. In PyTorch, you can access the gradients for the nodes in the computational graph by using the .grad member variable. Optimizers use the .grad variable to update the values of the parameters.

## CUDA

So far, we have been allocating our tensors on the CPU memory. When doing linear algebra operations, it might make sense to utilize a GPU, if you have one. To use a GPU, you need to first allocate the tensor on the GPU’s memory. Access to the GPUs is via a specialized API called CUDA. The CUDA API was created by NVIDIA and is limited to use on only NVIDIA GPUs. PyTorch offers CUDA tensor objects that are indistinguishable in use from the regular CPU-bound tensors except for the way they are allocated internally.

PyTorch makes it very easy to create these CUDA tensors, transfering the tensor from the CPU to the GPU while maintaining its underlying type. The preferred method in PyTorch is to be device agnostic and write code that works whether it’s on the GPU or the CPU. In Example 1-16, we first check whether a GPU is available by using torch.cuda.is_available(), and retrieve the device name with `torch.device()`. Then, all future tensors are instantiated and moved to the target device by using the .to(device) method

In [43]:
import torch
print (torch.cuda.is_available())

False


In [44]:
# preferred method: device agnostic tensor instantiation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print (device)

cpu


In [45]:
x = torch.rand(3, 3).to(device)
describe(x)

Type: torch.FloatTensor
Shape/size: torch.Size([3, 3])
Values: 
tensor([[0.4007, 0.4162, 0.3739],
        [0.7940, 0.9117, 0.1933],
        [0.6031, 0.6931, 0.4383]])


## Exercises

The best way to master a topic is to solve problems. Here are some warm-up exercises. Many of the problems will require going through the official documentation and finding helpful functions.

1. Create a 2D tensor and then add a dimension of size 1 inserted at dimension 0.

In [50]:
x = torch.randn(2, 2)

x.unsqueeze(0)

tensor([[[ 0.1332,  1.3649],
         [-0.7581, -1.3060]]])

2. Remove the extra dimension you just added to the previous tensor.

In [51]:

x.squeeze()

tensor([[ 0.1332,  1.3649],
        [-0.7581, -1.3060]])


3. Create a random tensor of shape 5x3 in the interval [3, 7)

In [65]:
torch.randint(low=3, high=7, size=(5, 3))

tensor([[5, 4, 3],
        [3, 3, 6],
        [5, 3, 5],
        [6, 5, 5],
        [4, 3, 6]])

4. Create a tensor with values from a normal distribution (mean=0, std=1)

In [66]:
torch.randn(5, 5)

tensor([[ 0.0429, -1.5412, -0.1824,  0.7215, -0.4424],
        [-1.0638,  1.4793,  0.5950, -0.2643,  0.2648],
        [-0.7342,  1.6660, -1.6582, -1.2074, -0.4851],
        [-0.0735, -0.3999,  1.3311, -0.1567,  3.1149],
        [ 0.9817,  0.1725, -1.0549, -1.8220,  0.7943]])

5. Retrieve the indexes of all the nonzero elements in the tensor torch.Tensor([1, 1, 1, 0, 1])

In [70]:
x = torch.Tensor([1, 1, 1, 0, 1])

x != 0

tensor([1, 1, 1, 0, 1], dtype=torch.uint8)

6. Create a random tensor of size (3,1) and then horizontally stack four copies together.

In [82]:
x = torch.randn(3, 1)

torch.stack([x, x, x, x], dim=1).squeeze()

tensor([[-1.4720, -1.4720, -1.4720, -1.4720],
        [-0.5335, -0.5335, -0.5335, -0.5335],
        [-2.0849, -2.0849, -2.0849, -2.0849]])